In [1]:
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
# https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3
# https://gist.github.com/fchollet/f35fbc80e066a49d65f1688a7e99f069

# https://github.com/jimgoo/caffe-oxford102

#https://medium.com/nanonets/nanonets-how-to-use-deep-learning-when-you-have-limited-data-f68c0b512cab

In [19]:
from tensorflow.python.client import device_lib
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Activation, Dropout, Flatten, Dense
import matplotlib.pyplot as plt
import numpy as np
import os

import tensorflow
from time import time


In [17]:

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15189459813114540907
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 160628736
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14443424963375011593
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


# Load VGG model and weights

In [5]:
img_width, img_height = 224, 224

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

#model = applications.VGG19(weights = "imagenet", include_top=False)


In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

## Get bottleneck features

In [21]:
# dimensions of our images.
img_width, img_height = 224, 224  # 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = '/data/oxford102/train'
validation_data_dir = '/data/oxford102/val'
nb_train_samples = 5698
nb_validation_samples = 2441
epochs = 50
batch_size = 16

In [8]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
def extract_bottleneck_features()
    generator = datagen.flow_from_directory(
            train_data_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode=None,  # this means our generator will only yield batches of data, no labels
            shuffle=False)  # our data will be in order, so all first 1000 images will be cats, then 1000 dogs

    # the predict_generator method returns the output of a model, given
    # a generator that yields batches of numpy data
    bottleneck_features_train = model.predict_generator(generator, 5698)
    # save the output as a Numpy array
    np.save(open('bottleneck_features_vgg_oxford102_train.npy', 'w'), bottleneck_features_train)

    generator = datagen.flow_from_directory(
            validation_data_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode=None,
            shuffle=False)
    bottleneck_features_validation = model.predict_generator(generator, 2441)
    np.save(open('bottleneck_features_vgg_oxford102_train.npy', 'wb'), bottleneck_features_train)

    np.save(open('bottleneck_features_vgg_oxford102_validation.npy', 'wb'), bottleneck_features_validation)

In [ ]:
print("Done")

## load our saved data and train a small fully-connected model:



In [63]:
class_labels = ['pink primrose', 'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea', 'english marigold', 'tiger lily', 'moon orchid', 'bird of paradise', 'monkshood', 'globe thistle', 'snapdragon', "colt's foot", 'king protea', 'spear thistle', 'yellow iris', 'globe-flower', 'purple coneflower', 'peruvian lily', 'balloon flower', 'giant white arum lily', 'fire lily', 'pincushion flower', 'fritillary', 'red ginger', 'grape hyacinth', 'corn poppy', 'prince of wales feathers', 'stemless gentian', 'artichoke', 'sweet william', 'carnation', 'garden phlox', 'love in the mist', 'mexican aster', 'alpine sea holly', 'ruby-lipped cattleya', 'cape flower', 'great masterwort', 'siam tulip', 'lenten rose', 'barbeton daisy', 'daffodil', 'sword lily', 'poinsettia', 'bolero deep blue', 'wallflower', 'marigold', 'buttercup', 'oxeye daisy', 'common dandelion', 'petunia', 'wild pansy', 'primula', 'sunflower', 'pelargonium', 'bishop of llandaff', 'gaura', 'geranium', 'orange dahlia', 'pink-yellow dahlia?', 'cautleya spicata', 'japanese anemone', 'black-eyed susan', 'silverbush', 'californian poppy', 'osteospermum', 'spring crocus', 'bearded iris', 'windflower', 'tree poppy', 'gazania', 'azalea', 'water lily', 'rose', 'thorn apple', 'morning glory', 'passion flower', 'lotus', 'toad lily', 'anthurium', 'frangipani', 'clematis', 'hibiscus', 'columbine', 'desert-rose', 'tree mallow', 'magnolia', 'cyclamen ', 'watercress', 'canna lily', 'hippeastrum ', 'bee balm', 'ball moss', 'foxglove', 'bougainvillea', 'camellia', 'mallow', 'mexican petunia', 'bromelia', 'blanket flower', 'trumpet creeper', 'blackberry lily']
lb = np.loadtxt('/data/oxford102/imagelabels.csv', dtype=int)
def get_class_labels():
    np.random.seed(42)
    xx = np.arange(len(lb))  
    y = np.random.choice(xx, size=int(len(xx)*0.3), replace=False)
    X = np.array(list(set(set(xx) - set(y))))
    #train_labels = [class_labels[lb[i]-1] for i in X]
    #val_labels = [class_labels[lb[i]-1] for i in y]
    train_labels = np.array([lb[i] for i in X])
    val_labels = np.array([lb[i] for i in y])
    return train_labels, val_labels

In [64]:
#train_data = np.load(open('bottleneck_features_vgg_oxford102_train.npy', "rb"))
#validation_data = np.load(open('bottleneck_features_vgg_oxford102_validation.npy', "rb"))
train_labels, validation_labels = get_class_labels()

In [106]:
model = Sequential()
# model.add(Flatten(input_shape=train_data.shape[1:]))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='sigmoid'))
model.add(Flatten(input_shape=train_data.shape[1:]))  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(102, activation="softmax"))

In [107]:
validation_data.shape

(38951, 7, 7, 512)

In [108]:
validation_labels.shape

(2456,)

In [109]:
train_data.shape

(90958, 7, 7, 512)

In [110]:
train_labels.shape

(5733,)

In [111]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_14 (Flatten)         (None, 25088)             0         
_________________________________________________________________
dense_25 (Dense)             (None, 256)               6422784   
_________________________________________________________________
activation_12 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 102)               26214     
Total params: 6,448,998
Trainable params: 6,448,998
Non-trainable params: 0
_________________________________________________________________


In [112]:


# TODO: create labels array
# the features were saved in order, so recreating the labels is easy
#validation_labels = np.array([0] * 400 + [1] * 400)


model.compile(optimizer='rmsprop', # optimizers.SGD(lr=0.0001, momentum=0.9)
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=2,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights('bottleneck_fc_model.h5')

ValueError: Error when checking target: expected dense_26 to have shape (102,) but got array with shape (1,)

# Fine-tuning the top layers of a a pre-trained network

In [113]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [135]:
y_train= np.zeros((102,), dtype='float32')

In [136]:
y_train_=OneHotEncoded(y_train)

In [117]:

%%matplotlibmatplot  inline
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm
from sklearn.metrics import log_loss, accuracy_score
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

UsageError: Cell magic `%%matplotlibmatplot` not found.


In [137]:
def OneHotEncoded(y_train):
    y_t=np.zeros((len(y_train),Num_Class), dtype=int)
    for i,x in enumerate(y_train):
        y_t[i][int(x)-1]=1
    return y_t

In [138]:
Num_Class=102

In [139]:
y_train_

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [131]:
len(train_labels)

5733

# Use bottleneck features to train a SVM or logistic regression model